# SQL-Genie — Phase 1: Fine-Tuning Brain
### Llama 3.1 8B × Unsloth × Liger Kernels × 400k Dataset

**Runtime target:** Google Colab A100 (40GB)

**Pipeline:**
1. Install dependencies (Unsloth + Liger Kernels)
2. Authenticate with HuggingFace
3. Load & preprocess the 3-dataset blend (FineTome-100k + Gretel SQL + UltraChat-200k)
4. Tokenize with Llama-3 chat template
5. Fine-tune with Unsloth (4-bit QLoRA) + Liger Kernels
6. Evaluate on SQL holdout samples
7. Export to GGUF (Q4_K_M) for Phase 2 deployment

---
>  **Before running:** Set your `HF_TOKEN` in *Colab Secrets* (icon in the left sidebar). You'll need access to `meta-llama/Llama-3.1-8B`.

## Install Dependencies
Unsloth's installer handles the CUDA-matched torch/triton versions automatically.

In [10]:
!pip uninstall -y huggingface_hub
!pip install --upgrade "huggingface_hub>=0.23.0"
!pip install --upgrade hf_transfer

Found existing installation: huggingface_hub 0.36.2
Uninstalling huggingface_hub-0.36.2:
  Successfully uninstalled huggingface_hub-0.36.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.3/596.3 kB 1.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.6 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.5.0 which is incompatible.


In [11]:
!pip uninstall -y unsloth liger-kernel trl transformers && pip cache purge
!pip install "unsloth[colab-new]" -q --no-cache-dir
!pip install liger-kernel -q --no-cache-dir
!pip install datasets trl transformers sentencepiece -q --no-cache-dir
!pip install llama-cpp-python -q --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124
print('\n All packages installed.')

Found existing installation: unsloth 2026.2.1
Uninstalling unsloth-2026.2.1:
  Successfully uninstalled unsloth-2026.2.1
Found existing installation: liger_kernel 0.7.0
Uninstalling liger_kernel-0.7.0:
  Successfully uninstalled liger_kernel-0.7.0
Found existing installation: trl 0.24.0
Uninstalling trl-0.24.0:
  Successfully uninstalled trl-0.24.0
Found existing installation: transformers 4.57.6
Uninstalling transformers-4.57.6:
  Successfully uninstalled transformers-4.57.6
Files removed: 8
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 938.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 432.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 626.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.3/432.3 kB 387.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

## Environment Check

In [12]:
# Core
import os
import glob
import time
import textwrap
import subprocess
import torch

# Hugging Face / Datasets
from datasets import load_dataset
from huggingface_hub import login

# Colab
from google.colab import userdata, files

# Unsloth / Training
import unsloth
from unsloth import FastLanguageModel
from liger_kernel.transformers import apply_liger_kernel_to_llama
from trl import SFTTrainer, SFTConfig
from google.colab import drive



In [13]:
# Mount Gdrive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [14]:
gpu_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'NO GPU DETECTED'
vram_gb  = torch.cuda.get_device_properties(0).total_memory / 1e9 if torch.cuda.is_available() else 0

print(f'GPU  : {gpu_name}')
print(f'VRAM : {vram_gb:.1f} GB')
print(f'CUDA : {torch.version.cuda}')

if vram_gb < 35:
    print('\n  WARNING: Less than 35 GB VRAM detected.')
    print('   Recommended runtime is A100 40GB.')
    print('   Runtime → Change runtime type → A100')
else:
    print('\n Hardware looks good — A100 confirmed!')


GPU  : NVIDIA RTX PRO 6000 Blackwell Server Edition
VRAM : 102.0 GB
CUDA : 12.8

 Hardware looks good — A100 confirmed!


## 2 Authenticate with HuggingFace

In [15]:
HF_TOKEN = userdata.get('HF_TOKEN')
login(token=HF_TOKEN)
print(' Logged in to HuggingFace Hub')


 Logged in to HuggingFace Hub


## 3 Configuration — All Hyperparameters in One Place

In [16]:
MODEL_ID        = 'meta-llama/Llama-3.1-8B-Instruct'
OUTPUT_DIR      = 'sql-genie-lora'
GGUF_OUTPUT_DIR = 'sql-genie-gguf'

# ── QLoRA ──────────────────────────────────────────────────────────────────
LORA_R          = 128     # sweet spot for 8B, frees VRAM for bigger batch
LORA_ALPHA      = 128
LORA_DROPOUT    = 0.0     # keep off — large batch handles regularization naturally
TARGET_MODULES  = ['q_proj','k_proj','v_proj','o_proj','gate_proj','up_proj','down_proj']

# ── Training ───────────────────────────────────────────────────────────────
MAX_SEQ_LENGTH  = 1024    # quadratic cost, was causing 159hr estimate
BATCH_SIZE      = 128      #  linear cost = fast
GRAD_ACCUM      = 2
LEARNING_RATE   = 8e-4
WARMUP_RATIO    = 0.05
NUM_EPOCHS      = 1
SAVE_STEPS      = 200
LOG_STEPS       = 25
SEED            = 42

# ── Dataset ────────────────────────────────────────────────────────────────
FINETOME_SAMPLES  = 100_000
SQL_SAMPLES       = 100_000
ULTRACHAT_SAMPLES = 200_000

# Define the path in Google Drive to save the model
gdrive_path = "/content/gdrive/MyDrive/models/Infra"

print(' Config loaded.')
print(f'   Model        : {MODEL_ID}')
print(f'   LoRA rank    : {LORA_R}')
print(f'   Eff. batch   : {BATCH_SIZE * GRAD_ACCUM}')
print(f'   Max seq len  : {MAX_SEQ_LENGTH}')


 Config loaded.
   Model        : meta-llama/Llama-3.1-8B-Instruct
   LoRA rank    : 128
   Eff. batch   : 256
   Max seq len  : 1024


## 4 Load & Preprocess Datasets
Normalizes all 3 sources into unified ChatML format.

In [17]:
def standardize_format(example, source):
    messages = []
    if source == 'sql':
        system_content = 'You are an expert SQL engineer. Generate accurate, efficient SQL queries based on the user request and context provided.'
        if example.get('sql_context'):
            system_content += f" Schema context: {example['sql_context']}"
        messages.append({'role': 'system',    'content': system_content})
        messages.append({'role': 'user',      'content': example.get('sql_prompt', '')})
        messages.append({'role': 'assistant', 'content': example.get('sql', '')})
    return {'messages': messages}

print('Loading Gretel SQL...')
ds_sql = load_dataset('gretelai/synthetic_text_to_sql', split='train')
print(f'   Full dataset: {len(ds_sql):,} samples')

ds_sql = ds_sql.map(lambda x: standardize_format(x, 'sql'), remove_columns=ds_sql.column_names, num_proc=2)

split = ds_sql.train_test_split(test_size=0.10, seed=SEED)
train_dataset = split['train']
eval_dataset  = split['test']

print(f'\n Dataset ready!')
print(f'   Train : {len(train_dataset):,} samples')
print(f'   Eval  : {len(eval_dataset):,} samples')


Loading Gretel SQL...
   Full dataset: 100,000 samples

 Dataset ready!
   Train : 90,000 samples
   Eval  : 10,000 samples


## 5 Load Model with Unsloth (4-bit QLoRA) + Liger Kernels

In [18]:
apply_liger_kernel_to_llama()
print(' Liger Kernels applied to Llama architecture')

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name    = MODEL_ID,
    max_seq_length= MAX_SEQ_LENGTH,
    dtype         = None,
    load_in_4bit  = True,
    token         = HF_TOKEN,
)
print(f'\n Base model loaded: {MODEL_ID}')

model = FastLanguageModel.get_peft_model(
    model,
    r                = LORA_R,
    lora_alpha       = LORA_ALPHA,
    lora_dropout     = LORA_DROPOUT,
    target_modules   = TARGET_MODULES,
    bias             = 'none',
    use_gradient_checkpointing = 'unsloth',
    random_state     = SEED,
    use_rslora       = True,
)

total_params     = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'\n LoRA adapter attached')
print(f'   Trainable params : {trainable_params/1e6:.1f}M ({100*trainable_params/total_params:.2f}%)')
print(f'   Total params     : {total_params/1e6:.0f}M')

 Liger Kernels applied to Llama architecture
==((====))==  Unsloth 2026.2.1: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    NVIDIA RTX PRO 6000 Blackwell Server Edition. Num GPUs = 1. Max memory: 94.971 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 12.0. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.35. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

 Base model loaded: meta-llama/Llama-3.1-8B-Instruct

 LoRA adapter attached
   Trainable params : 335.5M (6.76%)
   Total params     : 4964M


## 6 Tokenize with Llama-3 Chat Template

In [19]:
tokenizer.chat_template = (
    '{% for message in messages %}'
    '{% if loop.first and messages[0]["role"] == "system" %}'
    '{{ "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n" + message["content"] + "<|eot_id|>" }}'
    '{% elif message["role"] == "user" %}'
    '{{ "<|start_header_id|>user<|end_header_id|>\n\n" + message["content"] + "<|eot_id|>" }}'
    '{% elif message["role"] == "assistant" %}'
    '{{ "<|start_header_id|>assistant<|end_header_id|>\n\n" + message["content"] + "<|eot_id|>" }}'
    '{% endif %}'
    '{% endfor %}'
    '{{ "<|start_header_id|>assistant<|end_header_id|>\n\n" }}'
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_fn(batch):
    texts = [
        tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=False)
        for msgs in batch['messages']
    ]
    return tokenizer(texts, truncation=True, max_length=MAX_SEQ_LENGTH, padding='max_length')

print('Tokenizing train split...')
tokenized_train = train_dataset.map(tokenize_fn, batched=True, batch_size=500, remove_columns=train_dataset.column_names, num_proc=2)
print('Tokenizing eval split...')
tokenized_eval  = eval_dataset.map(tokenize_fn, batched=True, batch_size=500, remove_columns=eval_dataset.column_names, num_proc=2)

sample = tokenizer.decode(tokenized_train[0]['input_ids'][:200])
print('\n--- Sample (first 200 tokens) ---')
print(sample)
print('\n Tokenization complete')


Tokenizing train split...
Tokenizing eval split...

--- Sample (first 200 tokens) ---
<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert SQL engineer. Generate accurate, efficient SQL queries based on the user request and context provided. Schema context: CREATE TABLE AircraftMaintenance (company TEXT, maintenance_date DATE, maintenance_cost FLOAT); INSERT INTO AircraftMaintenance (company, maintenance_date, maintenance_cost) VALUES ('AeroTech Inc.', '2022-01-10', 200000), ('AeroTech Inc.', '2022-03-15', 300000), ('AeroTech Inc.', '2022-03-30', 150000);<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the total spent on aircraft maintenance by AeroTech Inc. in Q1 2022?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

SELECT SUM(maintenance_cost) FROM AircraftMaintenance WHERE company = 'AeroTech Inc.' AND maintenance_date BETWEEN '2022-01-01' AND '2022-03-31';<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|finetun

## 7 Training

In [9]:
sft_config = SFTConfig(
    output_dir                  = gdrive_path,
    dataset_text_field          = 'text',
    max_seq_length              = MAX_SEQ_LENGTH,
    per_device_train_batch_size = BATCH_SIZE,
    gradient_accumulation_steps = GRAD_ACCUM,
    learning_rate               = LEARNING_RATE,
    lr_scheduler_type           = 'cosine',
    warmup_ratio                = WARMUP_RATIO,
    bf16                        = True,
    fp16                        = False,
    optim                       = 'adamw_8bit',
    dataloader_num_workers  = 4,    # parallel data loading
    dataloader_pin_memory   = True, # faster CPU→GPU transfer
    logging_steps               = LOG_STEPS,
    save_strategy               = 'steps',
    save_steps                  = SAVE_STEPS,
    eval_strategy               = 'steps',
    eval_steps                  = SAVE_STEPS,
    num_train_epochs            = NUM_EPOCHS,
    seed                        = SEED,
    report_to                   = 'none',
    gradient_checkpointing      = True,
)

trainer = SFTTrainer(
    model         = model,
    args          = sft_config,
    train_dataset = tokenized_train,
    eval_dataset  = tokenized_eval,
    tokenizer     = tokenizer,
)

print(' Launching training...')
print(f'   Effective batch size : {BATCH_SIZE * GRAD_ACCUM}')
print(f'   Steps per epoch      : ~{len(tokenized_train) // (BATCH_SIZE * GRAD_ACCUM):,}')

start = time.time()
trainer_stats = trainer.train()
elapsed = time.time() - start

print(f'\n Training complete in {elapsed/3600:.2f} hours')
print(f'   Final train loss : {trainer_stats.training_loss:.4f}')


🦥 Unsloth: Padding-free auto-enabled, enabling faster training.
 Launching training...
   Effective batch size : 192
   Steps per epoch      : ~468


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 90,000 | Num Epochs = 1 | Total steps = 469
O^O/ \_/ \    Batch size per device = 96 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (96 x 2 x 1) = 192
 "-____-"     Trainable parameters = 335,544,320 of 8,365,805,568 (4.01% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
200,6.031800,5.999296


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


Step,Training Loss,Validation Loss
200,6.031800,5.999296
400,5.866100,5.849271



 Training complete in 4.58 hours
   Final train loss : 6.4407


## 8 Evaluation — SQL Quality Check

In [20]:
FastLanguageModel.for_inference(model)

SQL_TEST_CASES = [
    {'schema': 'Table: orders(order_id, customer_id, amount, created_at)',
     'query' : 'Get total sales per customer for the last 30 days, ordered by highest spend.'},
    {'schema': 'Tables: employees(emp_id, name, dept_id, salary), departments(dept_id, dept_name)',
     'query' : 'Find average salary per department, only for departments with more than 5 employees.'},
    {'schema': 'Table: products(product_id, name, category, price, stock_quantity)',
     'query' : 'Which product categories have more than 100 total units in stock? Show count per category.'},
]

def run_sql_inference(schema, query, max_new_tokens=300):
    messages = [
        {'role': 'system', 'content': f'You are an expert SQL engineer. Schema context: {schema}'},
        {'role': 'user',   'content': query},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, temperature=0.1, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    generated = outputs[0][inputs.input_ids.shape[1]:]
    return tokenizer.decode(generated, skip_special_tokens=True).strip()

print('=' * 70)
print('SQL GENERATION EVALUATION')
print('=' * 70)
for i, tc in enumerate(SQL_TEST_CASES, 1):
    print(f'\n[Test {i}]')
    print(f'  Schema : {tc["schema"]}')
    print(f'  Query  : {tc["query"]}')
    response = run_sql_inference(tc['schema'], tc['query'])
    print(f'  Output :\n{textwrap.indent(response, "    ")}')
    print('-' * 70)


SQL GENERATION EVALUATION

[Test 1]
  Schema : Table: orders(order_id, customer_id, amount, created_at)
  Query  : Get total sales per customer for the last 30 days, ordered by highest spend.
  Output :
    Here is the SQL query to get the total sales per customer for the last 30 days, ordered by highest spend:

    ```sql
    SELECT 
      customer_id, 
      SUM(amount) as total_sales
    FROM 
      orders
    WHERE 
      created_at >= NOW() - INTERVAL 30 DAY
    GROUP BY 
      customer_id
    ORDER BY 
      total_sales DESC;
    ```

    This query works as follows:

    1. `created_at >= NOW() - INTERVAL 30 DAY` filters the orders to only include those created within the last 30 days.
    2. `SUM(amount)` calculates the total sales for each customer.
    3. `GROUP BY customer_id` groups the results by customer ID.
    4. `ORDER BY total_sales DESC` sorts the results by the total sales in descending order, so the customers with the highest spend are at the top.

    Note: The `N

## 9 Save LoRA Adapter

In [21]:
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f' LoRA adapter saved to ./{OUTPUT_DIR}')

# Optional: push to HuggingFace Hub
# HF_USERNAME = 'YOUR_HF_USERNAME'
# REPO_NAME   = 'sql-genie-llama-3.1-8b-lora'
# model.push_to_hub(f'{HF_USERNAME}/{REPO_NAME}', token=HF_TOKEN)
# tokenizer.push_to_hub(f'{HF_USERNAME}/{REPO_NAME}', token=HF_TOKEN)


 LoRA adapter saved to ./sql-genie-lora


In [22]:

model.save_pretrained(gdrive_path)
tokenizer.save_pretrained(gdrive_path)
print(f' LoRA adapter saved to ./{OUTPUT_DIR}')
# Save the model's state dictionary to Google Drive
# torch.save(model.state_dict(), gdrive_path)
# print(f"Model saved to Google Drive: {gdrive_path}")

 LoRA adapter saved to ./sql-genie-lora


## 10 Export to GGUF (Q4_K_M)

| Format | Size | Quality | Best for |
|--------|------|---------|----------|
| `q4_k_m` | ~4.8 GB | ⭐⭐⭐⭐ | Phase 2 deployment ← **recommended** |
| `q8_0`   | ~8.5 GB | ⭐⭐⭐⭐⭐ | Highest quality |
| `f16`    | ~16 GB  | ⭐⭐⭐⭐⭐ | Full precision |

In [23]:
os.makedirs(GGUF_OUTPUT_DIR, exist_ok=True)

print('Merging LoRA weights and exporting to GGUF (Q4_K_M)...')
print('This may take 5-10 minutes...')

model.save_pretrained_gguf(GGUF_OUTPUT_DIR, tokenizer, quantization_method='q4_k_m')

gguf_files = glob.glob(f'{GGUF_OUTPUT_DIR}/*.gguf')
for f in gguf_files:
    size_gb = os.path.getsize(f) / 1e9
    print(f' Exported: {f}  ({size_gb:.2f} GB)')


Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Merging LoRA weights and exporting to GGUF (Q4_K_M)...
This may take 5-10 minutes...
Unsloth: Merging model weights to 16-bit format...


config.json:   0%|          | 0.00/896 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

...

Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [01:17<00:00, 19.37s/it]


Unsloth: Merge process complete. Saved to `/content/sql-genie-gguf`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF bf16 might take 3 minutes.
\        /    [2] Converting GGUF bf16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
Unsloth: Successfully installed llama.cpp!
Unsloth: Preparing converter script...


Unsloth: [1] Converting model into bf16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['sql-genie-gguf_gguf/llama-3.1-8b-instruct.BF16.gguf']
Unsloth: [2] Converting GGUF bf16 into q4_k_m. This might take 10 minutes...
Unsloth: Model files cleanup...


Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: All GGUF conversions completed successfully!
Generated files: ['sql-genie-gguf_gguf/llama-3.1-8b-instruct.Q4_K_M.gguf']
Unsloth: example usage for text only LLMs: llama.cpp/llama-cli --model sql-genie-gguf_gguf/llama-3.1-8b-instruct.Q4_K_M.gguf -p "why is the sky blue?"
Unsloth: Saved Ollama Modelfile to sql-genie-gguf_gguf/Modelfile
Unsloth: convert model to ollama format by running - ollama create model_name -f sql-genie-gguf_gguf/Modelfile


## 11 Download Artifacts

In [24]:
print('Zipping LoRA adapter...')
!zip -r sql-genie-lora.zip {OUTPUT_DIR}/
files.download('sql-genie-lora.zip')

gguf_files = glob.glob(f'{GGUF_OUTPUT_DIR}/*.gguf')
if gguf_files:
    print(f'Downloading GGUF: {gguf_files[0]}')
    print('  Large file — consider Hub push instead for reliability.')
    files.download(gguf_files[0])
else:
    print('No GGUF file found — run cell 10 first.')


Zipping LoRA adapter...
  adding: sql-genie-lora/ (stored 0%)
  adding: sql-genie-lora/adapter_config.json (deflated 57%)
  adding: sql-genie-lora/README.md (deflated 65%)
  adding: sql-genie-lora/special_tokens_map.json (deflated 71%)
  adding: sql-genie-lora/tokenizer.json (deflated 85%)
  adding: sql-genie-lora/tokenizer_config.json (deflated 96%)
  adding: sql-genie-lora/chat_template.jinja (deflated 65%)
  adding: sql-genie-lora/adapter_model.safetensors (deflated 57%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

No GGUF file found — run cell 10 first.
